# Problem Statment: 
download xml of all related paper from pmc



In [ ]:
!pip install BeautifulSoup4
!pip install fastprogress  #tqdm

In [ ]:
%matplotlib inline
import pandas as pd
import requests
from urllib.request import urlopen
from fastprogress.fastprogress import master_bar, progress_bar
from urllib.parse import quote
from bs4 import BeautifulSoup
import os
import time
from importlib import reload  # Not needed in Python 2
import logging
#reload(logging)
#logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.DEBUG, datefmt='%I:%M:%S')
logging.basicConfig(level=logging.DEBUG, datefmt='%I:%M:%S')
logger = logging.getLogger("Parsing_Data")
# logger = logging.getLogger("Parsing_Data")
# logger.setLevel(logging.DEBUG)

DEBUG:matplotlib.pyplot:Loaded backend module://ipykernel.pylab.backend_inline version unknown.


In [ ]:
# global variables
base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/"
search_url = "esearch.fcgi?db=pubmed&term="
fetch_url = "efetch.fcgi?db=pmc"
application = "&tool=digital_public_health"
email = "&email=siliang.liu@alumni.ubc.ca"

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
root_path = '/content/drive/My Drive/RA'

# key words:

Loading in key words and taking a look:

In [ ]:
path = "./search_term_1.csv"
search_terms_df = pd.read_csv(path, header=None, keep_default_na=False)
search_terms_df

In [ ]:
search_terms_c1 = search_terms_df.iloc[:,0]
search_terms_c2 = search_terms_df.iloc[:,1]

# Define search query

Let's define the query: we will use OR operator to connect all terms in column 0, as well as column 1, and use AND operator to connect subquery of column 0 and column 1. 

For instance, if we want to search ("virtual health" OR "mobile health") AND ("public health" OR "health promotion"), we would use query: "virtual+health[tw] OR mobile+health[tw]) AND (public+health[tw] OR health+promotion[tw]". 

Note: [tw] is a "Field Tag" helping to search specific fields of the PubMed, it refers to "text words"[1].

First, let's build the query

In [ ]:
def form_query_term(df):
    '''
    arg: df: dataframe containing a list of terms1 in single column
    return:
        array: containing processed search terms, eg: "mhealth[tw] OR m-health[tw] OR ehealth[tw]"
    '''
    query_arr = []
    tag = "[tw]"

    for term in df:
        #if term == "" or "*" in term:
        if term == "" in term:
            continue
            
        # eg: mhealth or m-health
        if " or " in term:
            or_terms = term.split(" or ")
            for each in or_terms:
               # query_string += each.rstrip().lstrip()
                query_arr.append(each.rstrip().lstrip().replace(" ", "+") + tag)
                
        else:
            query_arr.append(term.rstrip().lstrip().replace(" ", "+") + tag)
            
    return query_arr


In [ ]:
query_string = "(“mhealth” OR “m-health“ OR “ehealth“ OR “e-health“ OR “virtual health“ OR “mobile health“ OR “online health“ OR “internet-based health“ OR “computer-based health“ OR “health informatics“ OR “social media“ OR “predictive algorithms“ OR “artificial intelligence“ OR “machine learning methods“ OR “big data“ OR “electronic health“ OR “telemedicine“ OR “digit*” OR “web*”) AND (“public health“ OR “health promotion“ OR “health prevention“ OR “health+protection“ OR “health policy” OR “health determinants“ OR “health evaluation“ OR “health economics“ OR “public health ethics“ OR “risk assessment“ OR “epidemiology“ OR “community health“ OR “emergency preparedness“ OR “emergency response“ OR “health equity“ OR “social justice“ OR “social determinants“ OR “surveillance“) AND “last 20 years”[dp] AND “english”[la]"
# connector = " " + "OR" + " "
# query_arr_a = form_query_term(search_terms_c1)     
# query_arr_b = form_query_term(search_terms_c2) 
# query_string = "(" + connector.join(query_arr_a) + ")" + " AND " + "(" + connector.join(query_arr_b) + ")"
logger.debug(query_string)

DEBUG:Parsing_Data:(“mhealth” OR “m-health“ OR “ehealth“ OR “e-health“ OR “virtual health“ OR “mobile health“ OR “online health“ OR “internet-based health“ OR “computer-based health“ OR “health informatics“ OR “social media“ OR “predictive algorithms“ OR “artificial intelligence“ OR “machine learning methods“ OR “big data“ OR “electronic health“ OR “telemedicine“ OR “digit*” OR “web*”) AND (“public health“ OR “health promotion“ OR “health prevention“ OR “health+protection“ OR “health policy” OR “health determinants“ OR “health evaluation“ OR “health economics“ OR “public health ethics“ OR “risk assessment“ OR “epidemiology“ OR “community health“ OR “emergency preparedness“ OR “emergency response“ OR “health equity“ OR “social justice“ OR “social determinants“ OR “surveillance“) AND “last 20 years”[dp] AND “english”[la]


Secondly, Lets extract all the related ids givne the query

In [ ]:
def extract_ids(query_term, return_size=10):
    '''
    args: query_string 
    return: find and set max_return_size 
            return list of related ids
    '''

    ret_max_term = "&RetMax=" + str(return_size)
    
    base_url_key_word = base_url + search_url + quote(query_term) + ret_max_term #+ application + email
    #print(base_url_key_word)
    
    page_kw = urlopen(base_url_key_word)
    soup_kw = BeautifulSoup(page_kw, "xml")
    
    # get text form of everything in the xml file
    # soup_kw.get_text()
    
    return_size = soup_kw.find('Count').text    
    id_list = soup_kw.find_all('Id')
    
    return_ids = []
    for each_id in id_list:
        # print(each_id.text)
        return_ids.append(each_id.text)
    
    return return_size, return_ids

In [ ]:
return_size = extract_ids(query_string)[0]
logger.debug("there are {} related papers".format(return_size))
return_ids = extract_ids(query_string, return_size)[1]

# # save the pmc_list in a file:
# f=open(root_path + '/pmc_list_en.txt','w')
# for id in return_ids:
#     f.write(id+'\n')
# f.close()
#logger.debug(len(return_ids))

DEBUG:Parsing_Data:there are 60750 related papers


# Extract article information in xml

In [ ]:
def id_convert(pmid):
    
    base_url_converter = "https://www.ncbi.nlm.nih.gov/pmc/utils/idconv/v1.0/"
    api_url = base_url_converter + "?ids=" + pmid + "&format=json"
    #print(api_url)
    response = requests.get(api_url)

    if response.status_code == 200:
        res = json.loads(response.content.decode('utf-8'))
  #print(res)
        if 'pmcid' not in res['records'][0]:
            print("this pm_id : {} is not valid in pmc".format(pmid))
            return None
            
        else:
            pmc_id = res['records'][0]['pmcid']
            print("Corresponding pmcid is: {}".format(pmc_id))
            return pmc_id

In [ ]:
# return_ids = []
# with open('./pmc_list_en.txt','r') as f:
#   line = f.read()
#   return_ids.append(line)
# f.close()

# #save the pmc_list in a file:
# f=open(root_path + '/valid_pmc_list10.txt','w')
# for id in valid_ids:
#     f.write(id+'\n')
# f.close()

In [ ]:
import json
valid_ids = []

for each in return_ids:
  res = id_convert(each)
  if res is not None:
    valid_ids.append(res)
print(len(valid_ids))  

#save the valid pmc_list in a file:
f=open(root_path + '/pmc_en_valid.txt','w')
for id in valid_ids:
    f.write(id+'\n')
f.close()


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
pmc_return_ids = []
with open(root_path + '/pmc_miss.txt','r') as f:
    for line in f:
        pmc_return_ids.append(line.strip())
f.close()
print(len(pmc_return_ids))

17


In [ ]:
from concurrent.futures import ThreadPoolExecutor
import threading
sem = threading.Semaphore(3)

In [ ]:
#sem = threading.Semaphore(3)
import os.path
from os import path
def extract_xml(paper_id):
    #print("curretn paper_id is : {}".format(paper_id))
    # check if exist
    file = root_path + '/data/{}_paper.xml'.format(paper_id)
    if str(path.isfile(file)) is True:
      return

    query_id = "&id=" + str(paper_id)
    base_url_content = base_url + fetch_url + query_id
    print(base_url_content)
    sem.acquire()
    try:
      response = requests.get(base_url_content)
      with open(root_path + '/data/{}_paper.xml'.format(paper_id), 'wb') as file:
          file.write(response.content)
      file.close()
    finally:
        sem.release()
    

    # get text form of everything in the xml file
    #soup.get_text()
    
    time.sleep(1)

In [ ]:
# extract_xml(3869227)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): eutils.ncbi.nlm.nih.gov:443


https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pmc&id=3869227


DEBUG:urllib3.connectionpool:https://eutils.ncbi.nlm.nih.gov:443 "GET /entrez/eutils/efetch.fcgi?db=pmc&id=3869227 HTTP/1.1" 200 None


In [ ]:
def num_cpus():
    try:
        return len(os.sched_getaffinity(0))
    except AttributeError:
        return os.cpu_count()
num_cpus()

4

In [ ]:
last = pmc_return_ids[26326:]
len(last)

2218

In [ ]:
start = time.time()
# with ThreadPoolExecutor(max_workers=num_cpus()//2) as executor:
with ThreadPoolExecutor(max_workers=3) as executor:
    results = executor.map(extract_xml, pmc_return_ids)
#     jobs = (executor.submit(process, chunk) for chunk in df_generator)
print('Total time taken: {}'.format(time.time() - start))


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): eutils.ncbi.nlm.nih.gov:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): eutils.ncbi.nlm.nih.gov:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): eutils.ncbi.nlm.nih.gov:443


https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pmc&id=6085020
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pmc&id=6137798
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pmc&id=5958358


DEBUG:urllib3.connectionpool:https://eutils.ncbi.nlm.nih.gov:443 "GET /entrez/eutils/efetch.fcgi?db=pmc&id=6137798 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://eutils.ncbi.nlm.nih.gov:443 "GET /entrez/eutils/efetch.fcgi?db=pmc&id=6085020 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://eutils.ncbi.nlm.nih.gov:443 "GET /entrez/eutils/efetch.fcgi?db=pmc&id=5958358 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): eutils.ncbi.nlm.nih.gov:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): eutils.ncbi.nlm.nih.gov:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): eutils.ncbi.nlm.nih.gov:443


https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pmc&id=6336070
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pmc&id=6348559
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pmc&id=6459636


DEBUG:urllib3.connectionpool:https://eutils.ncbi.nlm.nih.gov:443 "GET /entrez/eutils/efetch.fcgi?db=pmc&id=6348559 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://eutils.ncbi.nlm.nih.gov:443 "GET /entrez/eutils/efetch.fcgi?db=pmc&id=6336070 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://eutils.ncbi.nlm.nih.gov:443 "GET /entrez/eutils/efetch.fcgi?db=pmc&id=6459636 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): eutils.ncbi.nlm.nih.gov:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): eutils.ncbi.nlm.nih.gov:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): eutils.ncbi.nlm.nih.gov:443


...

https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pmc&id=6611101
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pmc&id=6836806
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pmc&id=6775929


DEBUG:urllib3.connectionpool:https://eutils.ncbi.nlm.nih.gov:443 "GET /entrez/eutils/efetch.fcgi?db=pmc&id=6836806 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://eutils.ncbi.nlm.nih.gov:443 "GET /entrez/eutils/efetch.fcgi?db=pmc&id=6775929 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://eutils.ncbi.nlm.nih.gov:443 "GET /entrez/eutils/efetch.fcgi?db=pmc&id=6611101 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): eutils.ncbi.nlm.nih.gov:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): eutils.ncbi.nlm.nih.gov:443


https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pmc&id=7002948
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pmc&id=7067084


DEBUG:urllib3.connectionpool:https://eutils.ncbi.nlm.nih.gov:443 "GET /entrez/eutils/efetch.fcgi?db=pmc&id=7002948 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://eutils.ncbi.nlm.nih.gov:443 "GET /entrez/eutils/efetch.fcgi?db=pmc&id=7067084 HTTP/1.1" 200 None


Total time taken: 9.303781509399414


In [ ]:
cur_path = root_path + "/data/"
def shellquote(s):
    return "'" + s.replace("'", "'\\''") + "'"
cur_path = shellquote(cur_path)

! ls {cur_path} | wc -l

ls: cannot open directory '/content/drive/My Drive/[JOB] RA/data/': Input/output error
0


In [ ]:
# from pathlib import Path
# count = 0
# cur_path = root_path + "/data/"
# for path in Path(cur_path).glob('*.xml'):
#     count += 1
# count


OSError: ignored


# Reference

[1]Search Field Tags:  https://www.nlm.nih.gov/bsd/disted/pubmedtutorial/020_710.html